In [1]:
import pandas as pd

## Import Data

In [2]:
temperature = pd.read_csv('../../data/temperature_nsw.csv')
temperature_missing = pd.read_csv('../../data/temperature_nsw_additional.csv')
demand = pd.read_csv('../../data/totaldemand_nsw.csv')

## Drop Redundant Columns For Each Dataset

In [3]:
temperature.drop(['LOCATION'], axis=1, inplace=True)
demand.drop(['REGIONID'], axis=1, inplace=True)

## Remove Duplicate DATETIME values in Temperature Dataset

In [4]:
temperature = temperature.drop([temperature.index[19006], temperature.index[34282], temperature.index[34299],
                                temperature.index[34302], temperature.index[38655], temperature.index[58293],
                                temperature.index[78276], temperature.index[97917], temperature.index[117699],
                                temperature.index[137200], temperature.index[157015], temperature.index[176797],
                                temperature.index[196230]])

## Append Missing Value Dataset To Temperature Dataset

In [5]:
temperature = temperature.append(temperature_missing, ignore_index=True)

## Convert Values In DATETIME Columns From Strings To Datetime Values

In [6]:
temperature['DATETIME'] = pd.to_datetime(temperature['DATETIME'], dayfirst=True)
demand['DATETIME'] = pd.to_datetime(demand['DATETIME'], dayfirst=True)

## Create Minutes Column

In [7]:
temperature['MINUTE'] = temperature['DATETIME'].dt.minute

## Drop Record In The Temperature Dataset If The Minute Is Not 0 Or 30

In [8]:
temperature = temperature.drop(temperature[(temperature['MINUTE'] != 0) & (temperature['MINUTE'] != 30)].index)

## Drop MINUTE Column So That Missing Values Are Only Filled In For The TEMPERATURE Column

In [9]:
temperature.drop(['MINUTE'], axis=1, inplace=True)

## Fill In Remaining Missing Values So That All Timestamps Are Included

In [10]:
temperature = temperature.set_index('DATETIME').resample(rule='30T').interpolate(method='time').reset_index()

## Round Temperature Values To One Decimal Place

In [11]:
temperature.TEMPERATURE = temperature.TEMPERATURE.round(1)

## Create Separate Columns For Year, Month, Day, Day Of Week, Hour and Minute

In [12]:
temperature['DAY'] = temperature['DATETIME'].dt.day
temperature['DAY_OF_WEEK'] = temperature['DATETIME'].dt.dayofweek
temperature['MONTH'] = temperature['DATETIME'].dt.month
temperature['YEAR'] = temperature['DATETIME'].dt.year
temperature['HOUR'] = temperature['DATETIME'].dt.hour
temperature['MINUTE'] = temperature['DATETIME'].dt.minute

## Create Separate Column For Weekday/Weekend

In [14]:
weekday = list(range(5))

weekend = []
for i, value in enumerate(temperature['DAY_OF_WEEK']):
    if value in weekday:
        weekend.append(0)
    else:
        weekend.append(1)

temperature['WEEKEND'] = weekend

## Create Separate Column For Time Of Day

In [15]:
early_morning = list(range(6))
late_morning = list(range(6, 12))
afternoon = list(range(12, 18))
evening = list(range(18, 24))

time_of_day_train = []
for i, value in enumerate(temperature['HOUR']):
    if value in early_morning:
        time_of_day_train.append(1)
    elif value in late_morning:
        time_of_day_train.append(2)
    elif value in afternoon:
        time_of_day_train.append(3)
    elif value in evening:
        time_of_day_train.append(4)
        
temperature['TIME_OF_DAY'] = time_of_day_train

## Create Separate Column For Season

In [16]:
summer = [1, 2, 12]
autumn = [3, 4, 5]
winter = [6, 7, 8]
spring = [9, 10, 11]

season_train = []
for i, value in enumerate(temperature['MONTH']):
    if value in summer:
        season_train.append(1)
    elif value in autumn:
        season_train.append(2)
    elif value in winter:
        season_train.append(3)
    elif value in spring:
        season_train.append(4)
        
temperature['SEASON'] = season_train

## Export Temperature Dataset With All Missing Values Filled

In [17]:
temperature.to_csv('temperature_nsw_all_values_v2.csv')

## Merge Temperature And Demand Datasets

In [18]:
merged = temperature.merge(demand, how='left', on='DATETIME')

In [19]:
merged

,DATETIME,TEMPERATURE,DAY,DAY_OF_WEEK,MONTH,YEAR,HOUR,MINUTE,WEEKEND,TIME_OF_DAY,SEASON,TOTALDEMAND
0,2010-01-01 00:00:00,23.1,1,4,1,2010,0,0,0,1,1,8038.00
1,2010-01-01 00:30:00,22.9,1,4,1,2010,0,30,0,1,1,7809.31
2,2010-01-01 01:00:00,22.6,1,4,1,2010,1,0,0,1,1,7483.69
3,2010-01-01 01:30:00,22.5,1,4,1,2010,1,30,0,1,1,7117.23
4,2010-01-01 02:00:00,22.5,1,4,1,2010,2,0,0,1,1,6812.03
...,...,...,...,...,...,...,...,...,...,...,...,...
196508,2021-03-17 22:00:00,19.7,17,2,3,2021,22,0,0,4,2,7419.77
196509,2021-03-17 22:30:00,19.5,17,2,3,2021,22,30,0,4,2,7417.91
196510,2021-03-17 23:00:00,19.1,17,2,3,2021,23,0,0,4,2,7287.32
196511,2021-03-17 23:30:00,18.8,17,2,3,2021,23,30,0,4,2,7172.39


## Export Temperature Dataset With All Missing Values Filled

In [20]:
merged.to_csv('merged_all_values_v2.csv', index=False)